In [ ]:
from core.utils import Tibanna
from core import ff_utils
from tasks import run_md5
from tasks import run_fastqc
from invoke import run
import time
from datetime import datetime

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately
env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

set_resp = ff_utils.get_metadata('experiment-set-replicates', connection=ff, frame='embedded')['@graph']

def form_hyp(id):
    hyp = '=HYPERLINK("https://data.4dnucleome.org/{0}","{0}")'.format(id)
    return hyp

In [ ]:
c_del = 0
title_ele = ['ID', 'Status', 'Project', 'Center PI', 'Lab', 'Description', 'Release Date', 'Experiment Type', 'Number of Experiments']
print "\t".join(title_ele)

for a_set in set_resp:
    my_id = form_hyp(a_set['accession'])
    my_status = a_set['status']
    if my_status == 'deleted':
        c_del += 1
        continue
    #print a_set

    my_lab = a_set['lab']['title']
    my_desc = a_set['description']
    my_rel_date = a_set.get('date_released',"NA")
    my_group = a_set['award']['project']
    my_pi_id = ff_utils.get_metadata(a_set['award']['uuid'], connection=ff).get('pi', "NA")
    center_pi_name = ff_utils.get_metadata(my_pi_id, connection=ff)['title']
    
    my_exp_type = a_set['experiments_in_set'][0]['experiment_type']
    my_num_exp = a_set['number_of_experiments']
    
    print_list = [my_id, my_status, my_group, center_pi_name, my_lab, my_desc, my_rel_date,my_exp_type, str(my_num_exp)]
    print "\t".join(print_list)

